# ECE/CS 434 | MP2: DUET
<br />
<nav>
    <span class="alert alert-block alert-warning">Due on Wednesday March 3 11:59PM on Gradescope</span>
   
</nav><br> 


## Objective
In this MP, you will:
- Implement DUET algorithm to separate a mixture of N voice signals from received from two microphones

---
## Problem Overview
Consider a problem of separating N sources ($S_1$, $S_2$, ... $S_N$) from recordings on 2 microphones ($R_1$ and $R_2$).
According to DUET algorithm, you will need to perform the following steps:

- Calculate the short-time Fourier transform of two received signals to get the time-frequency spectrograms
- Calculate the ratio of the two time-frequency spectrograms to get relative delay and attenuation
- Cluster the time-frequency bins in the 2D space spanned by relative delay and attenuation
- Recover the original N signals based on the clustering results

You can refer to the original DUET paper in ICASSP 2000: "Blind separation of disjoint orthogonal signals: demixing N sources from 2 mixtures" and this tutorial in Blind speech separation, 2007 - Springer: "The DUET blind source separation algorithm"

For the sake of easier clustering, the exact number of sources N will be provided to you.

You can assume there is no time-frequency bin collision for any two sources.

---
## Imports & Setup
To run the grading script of this MP, you will need to install the Python [SpeechRecognition](https://pypi.org/project/SpeechRecognition/) package. The SpeechRecognition package also requires the dependency [pocketsphinx](https://pypi.org/project/pocketsphinx/). You may directly use pip install to install both packages.
The following `code` cell, when run, imports the libraries you might need for this MP. Feel free to delete or import other commonly used libraries. Double check with the TA if you are unsure if a library is supported.

In [190]:
import numpy as np
import scipy.io.wavfile
import speech_recognition as sr

if __name__ == '__main__':
    import matplotlib.pyplot as plt
    plt.style.use("seaborn-v0_8") # This sets the matplotlib color scheme to something more soothing
    from IPython import get_ipython
    get_ipython().run_line_magic('matplotlib', 'inline')

# This function is used to format test results. You don't need to touch it.
def display_table(data):
    from IPython.display import HTML, display
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4><td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

### Sanity-check

Running the following code block verifies that the correct module versions are indeed being used. 

Try restarting the Python kernel (or Jupyter) if there is a mismatch even after intalling the correct version. This might happen because Python's `import` statement does not reload already-loaded modules even if they are updated.

In [191]:
if __name__ == '__main__':
    from IPython.display import display, HTML

    def printc(text, color):
        display(HTML("<text style='color:{};weight:700;'>{}</text>".format(color, text)))

    _requirements = [r.split("==") for r in open(
        "packages.txt", "r").read().split("\n")]

    import sys
    for (module, expected_version) in _requirements:
        try:
            if sys.modules[module].__version__ != expected_version:
                printc("[✕] {} version should to be {}, but {} is installed.".format(
                    module, expected_version, sys.modules[module].__version__), "#f44336")
            else:
                printc("[✓] {} version {} is correct.".format(
                    module, expected_version), "#4caf50")
        except:
            printc("[–] {} is not imported, skipping version check.".format(
                module), "#03a9f4")

---
## Your Implementation
Implement your localization algorithm in the function `duet_source_separation(mic_data_folder, NUM_SOURCES)`. Do **NOT** change its function signature. You are, however, free to define and use helper functions. 

Your implementation for `duet_source_separation` function should **NOT** output any plots or data. It should only return the user's calculated location.

In [192]:
import scipy.io.wavfile
import scipy.signal
import numpy as np
from sklearn.cluster import KMeans


def duet_source_separation(mic_data_folder, NUM_SOURCES):
    """DUET source separation algorithm.

    Args:
        mic_data_folder: name of folder (without a trailing slash) containing
                         two mic datafiles `0.wav` and `1.wav`.
        NUM_SOURCES: The number of sources to recover.

    Returns:
        NUM_SOURCES * recording_length numpy array, where NUM_SOURCES is the number of sources,
        and recording_length is the original length of the recording (in number of samples)
    """
    n_clusters = NUM_SOURCES
    # --- Step 1: Load Audio Files ---
    try:
        sr_0, mic_0 = scipy.io.wavfile.read(f"{mic_data_folder}/0.wav")
        sr_1, mic_1 = scipy.io.wavfile.read(f"{mic_data_folder}/1.wav")
    except FileNotFoundError:
        raise FileNotFoundError(f"Audio files not found in {mic_data_folder}.  Ensure '0.wav' and '1.wav' exist.")
    except ValueError as e:  # Catch ValueError from scipy.io.wavfile.read
        raise ValueError("Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format") from e
    except Exception as e:
        raise Exception(f"Error reading audio files: {e}")
    # Ensure sample rates are the same
    if sr_0 != sr_1:
        raise ValueError("Sample rates of the two microphones must be the same.")
    sr = sr_0  # Use the common sample rate

    def high_pass_filter(signal, cutoff_freq, sr, order=5):
        nyquist = 0.5 * sr
        normal_cutoff = cutoff_freq / nyquist
        b, a = scipy.signal.butter(order, normal_cutoff, btype='high', analog=False)
        return scipy.signal.lfilter(b, a, signal)
    
    mic_0 = high_pass_filter(mic_0, 20, sr)
    mic_1 = high_pass_filter(mic_1, 20, sr)
    # --- Step 1: Create Low Pass Filter ---
    def low_pass_filter(signal, cutoff_freq, sr, order=5):
        nyquist = 0.5 * sr
        normal_cutoff = cutoff_freq / nyquist
        b, a = scipy.signal.butter(order, normal_cutoff, btype='low', analog=False)
        return scipy.signal.lfilter(b, a, signal)

    cutoff_freq = 4000  # Cutoff frequency in Hz
    mic_0 = low_pass_filter(mic_0, cutoff_freq, sr)
    mic_1 = low_pass_filter(mic_1, cutoff_freq, sr)
    
    # --- Step 2: Calculate STFT ---
    n_fft = 1024  # Window size
    hop_length = n_fft // 2  # Hop size
    window = 'hann'  # Window type

    F, T, stft_0 = scipy.signal.stft(mic_0, sr, window=window, nperseg=n_fft, noverlap=n_fft - hop_length)
    F, T, stft_1 = scipy.signal.stft(mic_1, sr, window=window, nperseg=n_fft, noverlap=n_fft - hop_length)
    
    
    # Convert to mono if stereo
    if len(mic_0.shape) > 1:
        mic_0 = np.mean(mic_0, axis=1)
    if len(mic_1.shape) > 1:
        mic_1 = np.mean(mic_1, axis=1)

    recording_length = len(mic_0)  # Get length before normalization
    original_dtype = mic_0.dtype  # Store original data type

    # Normalize the signals to float32 for processing
    mic_0 = mic_0.astype(np.float32) / np.iinfo(original_dtype).max if np.issubdtype(original_dtype, np.integer) else mic_0.astype(np.float32)
    mic_1 = mic_1.astype(np.float32) / np.iinfo(original_dtype).max if np.issubdtype(original_dtype, np.integer) else mic_1.astype(np.float32)

    # Find the threshold of the top 10% magnitude of stft_0
    log_magnitude_stft_0 = np.log(np.abs(stft_0) + 1e-10)
    log_magnitude_stft_1 = np.log(np.abs(stft_1) + 1e-10)
    threshold = np.percentile(log_magnitude_stft_0, 95)
    #print(f"Threshold for top 10% magnitude of stft_0: {threshold}")

    delta_t = []
    delay_times = []
    for i in range(0,F.shape[0]):
        for j in range(0,T.shape[0]):
            
            frequency = F[i]
            phase_diff = np.angle(stft_0[i,j]) - np.angle(stft_1[i,j])
            dt = phase_diff / (2 * np.pi * frequency+1e-10)
            delay_times.append(dt)
            if(frequency>1000 or frequency<20): # remove aliasing
                continue
            if(np.log(np.abs(stft_0[i,j])) < threshold):
                continue
            if(abs(dt)>0.2/340):
                continue
            delta_t.append(dt)
    delta_t = np.array(delta_t)
    delay_times = np.array(delay_times)




    # --- Step 4: Perform K-Means Clustering ---
    # Cluster the time-frequency bins into N clusters

    # Cluster the delta_t values
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(delta_t.reshape(-1, 1))
    #print("Cluster centers:", kmeans.cluster_centers_)

    # using the cluster centers to predict the masks for the delay times
    masks = kmeans.predict(delay_times.reshape(-1, 1))
    masks = masks.reshape(stft_0.shape)
    #print(masks.shape)

    sources = []
    for k in range(n_clusters):
        # Apply mask to S_R1 to isolate bins for source k
        S_source_k = stft_0 * (masks==k)
        # Reconstruct the time-domain signal
        source_k = scipy.signal.istft(S_source_k, sr, window=window, nperseg=n_fft, noverlap=n_fft - hop_length)[1]
        sources.append(source_k)
        scipy.io.wavfile.write(f"{k}.wav", sr, source_k.astype(np.int16))
    
    
    output = np.array(sources).astype(np.int16)
    return output

---
## Running and Testing
Use the cell below to run and test your code, and to get an estimate of your grade.

In [193]:
def calculate_score(calculated, expected):
    student_result = set()
    calculated = np.array(calculated)
    if calculated.shape[0] != len(expected):
      print(calculated.shape[0], len(expected))
      return 0, {'Incorrect number of sources!'}
    for i in range(calculated.shape[0]):
        scipy.io.wavfile.write("temp.wav",22050,calculated[i,:])
        r = sr.Recognizer()
        with sr.AudioFile("temp.wav") as source:
            audio = r.record(source)
        try:
            text = r.recognize_sphinx(audio)
            student_result.add(text.lower())
        except:
            student_result.add("Sphinx could not understand audio")
    score = len(student_result.intersection(expected))/len(expected)
    return score, student_result
     
if __name__ == '__main__':
    groundtruth = [{"hello how are you"}, {"nice to meet you","how are you"}, {"how are you","good morning","nice to meet you"}]
    
    output = [['Dataset', 'Expected Output', 'Your Output', 'Grade', 'Points Awarded']]
    for i in range(1,4):
        directory_name = 'dataset{}'.format(i)
        student_output = duet_source_separation(directory_name, i)
        result = calculate_score(student_output, groundtruth[i-1])   
        output.append([
            str(i),
            str(groundtruth[i-1]), 
            str(result[1]), 
            "{:2.2f}%".format(result[0] * 100),
            "{:1.2f} / 5.0".format(result[0] * 5),
        ])

    output.append([
        '<i>👻 Hidden test 1 👻</i>', 
        '<i>???</i>', 
        '<i>???</i>', 
        '<i>???</i>', 
        "<i>???</i> / 10.0"])
    output.append([
        '<i>...</i>', 
        '<i>...</i>', 
        '<i>...</i>', 
        '<i>...</i>', 
        "<i>...</i>"])
    output.append([
        '<i>👻 Hidden test 7 👻</i>', 
        '<i>???</i>', 
        '<i>???</i>', 
        '<i>???</i>', 
        "<i>???</i> / 10.0"])
    display_table(output)

---
## Rubric
You will be graded on the three data points provided to you (5 points each) and seven additional data points under different settings(10 points each). We will use the same code from the **Running and Testing** section above to grade all 10 traces of data. We will run ASR on your output to see if it generates the corrected separated speech signal. Output order does not matter. Percentage of grade for each data point is based on how many sources you estimated correctly (i.e., assume there are n sources, then you will get $\frac{1}{n} * 100\%$ for each correctedly estimated source).

---
## Submission Guidlines
This Jupyter notebook (`MP2.ipynb`) is the only file you need to submit on Gradescope. As mentioned earlier, you will only be graded using your implementation of the `duet_source_separation` function, which should only return the calculated **NOT** output any plots or data. 

**Make sure any code you added to this notebook, except for import statements, is either in a function or guarded by `__main__`(which won't be run by the autograder). Gradescope will give you immediate feedback using the provided test cases. It is your responsibility to check the output before the deadline to ensure your submission runs with the autograder.**